In [1]:
import queue
import re
from urllib.parse import urlparse
from urllib.parse import urlsplit
from urllib.parse import urljoin
from scrapy import Selector
import csv
import time
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
#this is a faster list
from collections import deque
import os

In [2]:

new_urls = deque(['http://www.stevens.edu'])

#we use a set for urls to avoid duplicated 
processed_urls = set()

#list of all email address
emails = set()
#print(new_urls)

#process urls from queue on by on until we exhaust this queue
#for _ in range(10):
while len(new_urls):
    #move next url from the queue to the set of processed urls
    url = new_urls.popleft()
    #print("I got new url:",url)
    #resotre the current url to already done list
    processed_urls.add(url)
    #print("URL has benn processed:",processed_urls)
    #extract base url and path to resolve relative links
    #schema="https", netloc='www.stevens.edu', path='/school-business/faculty', query='', fragment=''
    parts = urlsplit(url)
    #print("After urlspli():",parts)
    base_url = "{0.scheme}://{0.netloc}".format(parts)
    #print("Base url is :", base_url)
    path = url[:url.rfind('/')+1] if '/' in parts.path else url
    #print("path is: ", path)
    #get url's content, if requests error, we esccap and proced next
    url_str = str(url)
    url_str = url_str.replace("https://","")
    url_str = url_str.replace("http://","")
    url_str = url_str.replace("?","")
    url_str = url_str.replace("\\","")
    url_str = url_str.replace("*","")
    url_str = url_str.replace("<","")
    url_str = url_str.replace(">","")
    url_str = url_str.replace(":","")
    url_str = url_str.replace("\"","")
    url_join = ('/root/'+ url_str)
    #print(url_str, url_join)
    os.makedirs(url_str, exist_ok=True)
    print("Processing %s" % url)
    #driver = webdriver.Chrome()
    #driver.get(url)
    try:
        response = requests.get(url)
    except requests.exceptions.ConnectionError:
        #ignore pages with errors
        continue
    #print(response.status_code, response.url)
    #When we got web content, we find all email in it and add it to list.
    soup = BeautifulSoup(response.text)
    #email_addresses += re.findall("\S+@stevens.edu", soup.get_text())
    new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@stevens.edu", soup.get_text(), re.I))

    print("new_emails are:",new_emails)
    emails.update(new_emails)
    print("we success update emails into emails",emails)
    #we find url in this webssit and add it to queue
    soup = BeautifulSoup(response.text)
    #find and process all tha anchors in this html
    for anchor in soup.find_all("a"):
        #extract link url from the anchor
        link = anchor.get("href")
        #link=anchor.attrs["href"] if "href" in anchor.attrs else ''
        #print("any href link:",link)
        #resolve relative links
        #with open(file_path, 'w+',encoding='utf-8-sig',newline='') as csvfile:
        #writer = csv.writer(csvfile)
        try:
            link.startswith('/')
            #link = base_url +link
            url_crawl = urljoin(base_url,link)
            new_urls.append(url_crawl)
            #writer.writerow([url_crawl,emails])
        except:
            url_crawl = link 
            new_urls.append(url_crawl)
            #writer.writerow([url_crawl,emails])

            
    #print("we get new link:",new_urls)
file_path='20_Assignment 03 Email_list.csv' 
with open("email.txt", 'w+',encoding='utf-8-sig',newline='') as csvfile:
    for i in emails:
        csvfile.write(i + "\n")

Processing http://www.stevens.edu
new_emails are: set()
we success update emails into emails set()
Processing http://www.stevens.edu#page
new_emails are: set()
we success update emails into emails set()
Processing http://www.stevens.edu/
new_emails are: set()
we success update emails into emails set()
Processing http://www.stevens.edu/admissions/undergraduate-admissions/how-apply
new_emails are: set()
we success update emails into emails set()
Processing https://gradadmissions.stevens.edu/apply/
new_emails are: {'graduate@stevens.edu'}
we success update emails into emails {'graduate@stevens.edu'}
Processing http://www.stevens.edu/give-now
new_emails are: set()
we success update emails into emails {'graduate@stevens.edu'}
Processing http://www.stevens.edu/learn-about-giving
new_emails are: set()
we success update emails into emails {'graduate@stevens.edu'}
Processing http://power.stevens.edu
new_emails are: set()
we success update emails into emails {'graduate@stevens.edu'}
Processing h

KeyboardInterrupt: 